🧩 **1️⃣ Importação das bibliotecas**  
Importamos as bibliotecas necessárias para o ETL:

- **pandas** → manipulação e análise de dados  
- **os** → interação com o sistema de arquivos  
- **sqlite3** → criar e gerenciar banco de dados SQLite


In [3]:
import pandas as pd
import os
import sqlite3

print("✅ Bibliotecas importadas")



✅ Bibliotecas importadas


🧩 **2️⃣ Preparação da pasta de dados**  
Garantimos que existe uma pasta `/content/data` para armazenar os arquivos CSV e o banco de dados.


In [4]:
os.makedirs("/content/data", exist_ok=True)
print("✅ Pasta /content/data pronta")


✅ Pasta /content/data pronta


🧩 **3️⃣ Carregamento do CSV**  
Carregamos o CSV gerado no primeiro notebook para iniciar o ETL.  
Caso o arquivo não exista, o Colab irá solicitar upload manual.


In [5]:
caminho_arquivo = "/content/data/vendas.csv"

if os.path.exists(caminho_arquivo):
    df = pd.read_csv(caminho_arquivo)
    print("✅ CSV carregado com sucesso!")
    display(df.head())
else:
    print("⚠️ Arquivo CSV não encontrado. Faça upload do arquivo.")
    from google.colab import files
    uploaded = files.upload()
    for fn in uploaded.keys():
        os.rename(fn, f"/content/data/{fn}")
    df = pd.read_csv(f"/content/data/{fn}")
    print("✅ CSV carregado com sucesso após upload!")
    display(df.head())


✅ CSV carregado com sucesso!


,ID_Venda,Produto,Categoria,Cidade,Vendedor,Quantidade,Preco_Unitario,Total_Venda,Data_Venda
0,32,Boné,Acessórios,Fortaleza,Patrícia,7,153.41,1073.87,2024-10-24
1,6,Boné,Acessórios,Salvador,Ana,9,493.45,4441.05,2024-10-30
2,47,Jaqueta,Vestuário,Belo Horizonte,João,3,445.31,1335.93,2024-11-02
3,19,Camisa,Vestuário,Fortaleza,Patrícia,7,50.78,355.46,2024-11-04
4,22,Camisa,Vestuário,Rio de Janeiro,Paulo,5,287.98,1439.90,2024-11-14


🧩 **4️⃣ Transformação dos dados**  
Nesta etapa aplicamos transformações para deixar os dados prontos para análise:

- Conversão de colunas numéricas  
- Conversão de datas  
- Remoção de duplicados  
- Tratamento de valores ausentes


In [6]:
df = df.copy()

# Exemplo de conversão de colunas numéricas
for col in df.columns:
    if 'preco' in col.lower() or 'valor' in col.lower() or 'quantidade' in col.lower():
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Conversão de colunas de datas
for col in df.columns:
    if 'data' in col.lower():
        df[col] = pd.to_datetime(df[col], errors='coerce')

# Remover duplicados
df = df.drop_duplicates()

# Mostrar resultado da transformação
print("✅ Transformações aplicadas")
display(df.head())


✅ Transformações aplicadas


,ID_Venda,Produto,Categoria,Cidade,Vendedor,Quantidade,Preco_Unitario,Total_Venda,Data_Venda
0,32,Boné,Acessórios,Fortaleza,Patrícia,7,153.41,1073.87,2024-10-24
1,6,Boné,Acessórios,Salvador,Ana,9,493.45,4441.05,2024-10-30
2,47,Jaqueta,Vestuário,Belo Horizonte,João,3,445.31,1335.93,2024-11-02
3,19,Camisa,Vestuário,Fortaleza,Patrícia,7,50.78,355.46,2024-11-04
4,22,Camisa,Vestuário,Rio de Janeiro,Paulo,5,287.98,1439.90,2024-11-14


🧩 **5️⃣ Carga no banco SQLite**  
Salvamos os dados transformados em um banco SQLite para consultas futuras.  
- **if_exists='replace'** → substitui a tabela se já existir  
- **index=False** → não salva o índice do DataFrame


In [7]:
# Conexão com o banco SQLite
conn = sqlite3.connect("/content/data/banco_vendas.db")

# Carregar DataFrame no banco
df.to_sql("vendas", conn, if_exists="replace", index=False)

print("✅ Dados carregados no banco SQLite com sucesso")


✅ Dados carregados no banco SQLite com sucesso


🧩 **6️⃣ Consulta de verificação no banco**  
Verificamos se os dados foram salvos corretamente realizando uma consulta simples.


In [8]:
query = "SELECT * FROM vendas LIMIT 5"
df_db = pd.read_sql(query, conn)
display(df_db)


,ID_Venda,Produto,Categoria,Cidade,Vendedor,Quantidade,Preco_Unitario,Total_Venda,Data_Venda
0,32,Boné,Acessórios,Fortaleza,Patrícia,7,153.41,1073.87,2024-10-24 00:00:00
1,6,Boné,Acessórios,Salvador,Ana,9,493.45,4441.05,2024-10-30 00:00:00
2,47,Jaqueta,Vestuário,Belo Horizonte,João,3,445.31,1335.93,2024-11-02 00:00:00
3,19,Camisa,Vestuário,Fortaleza,Patrícia,7,50.78,355.46,2024-11-04 00:00:00
4,22,Camisa,Vestuário,Rio de Janeiro,Paulo,5,287.98,1439.90,2024-11-14 00:00:00


🧩 **7️⃣ Finalização do ETL**  
Pipeline concluído com sucesso.  
O CSV transformado foi carregado no banco SQLite e está pronto para análise.


In [9]:
conn.close()
print("✅ Conexão com o banco encerrada. ETL finalizado!")


✅ Conexão com o banco encerrada. ETL finalizado!
